### Hello

In [2]:
!pip install torch==2.0.1 torchtext==0.15.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [27]:
!python -m spacy download de_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 24.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torchdata.datapipes.iter import IterableWrapper, Mapper
import torchtext

import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import random

In [6]:
import torchtext
print(torchtext.__version__)

0.15.2+cpu


### Dataset

A data set in PyTorch is an object that represents a collection of data samples. Each data sample consists of one or more input features and their  target labels.

### Dataloader
- For efficiently loading and batching data from a data set.
- Data loaders parameters, includes data set to load from, batch size (determining how many samples per batch), shuffle (whether to shuffle the data for each epoch), and more. Data loaders also provide an iterator interface, making it easy to iterate over batches of data during training.
- In PyTorch, not all data sets are iterators, but all data loaders are.
- Purpose is to convert input data and labels into batches of tensors with the same shape for deep learning models to interpret.
- It can be used for tasks such as tokenizing, sequencing, converting your samples to the same size, and transforming your data into tensors that your model can understand.

We can use custom data set called CustomDataset. This data set inherits from the `torch.utils.data.Dataset` class and is initialized with a list of sentences. This comprises of two essential methods:

- __init__(self, sentences): Initializes the data set with a list of sentences.
- __getitem__(self, idx): Retrieves an item (in this case, a sentence) at a specific index, idx.

In [11]:
sentences = [
    "If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals.",
    "Fame's a fickle friend, Harry.",
    "It is our choices, Harry, that show what we truly are, far more than our abilities.",
    "Soon we must all face the choice between what is right and what is easy.",
    "Youth can not know how age thinks and feels. But old men are guilty if they forget what it was to be young.",
    "You are awesome!"
]

#The name in parentheses (Dataset) specifies that the new class CustomDataset is inheriting all the properties and methods of the Dataset class.
#This is Python’s standard way of defining inheritance.
#Even though Dataset provides a general framework, the CustomDataset class overrides its behavior by implementing its own versions of __len__ and __getitem__ methods. These methods are customized to handle the sentences data.

# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, sentences):
        self.sentences = sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx]

# Create an instance of your custom dataset
custom_dataset = CustomDataset(sentences)


In [9]:
custom_dataset.__getitem__(1) ## second sentence in the list

"Fame's a fickle friend, Harry."

In [12]:
# Define batch size
batch_size = 2

# Create a DataLoader
dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Iterate through the DataLoader
for batch in dataloader:
    print(batch)

['You are awesome!', "If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals."]
["Fame's a fickle friend, Harry.", 'Youth can not know how age thinks and feels. But old men are guilty if they forget what it was to be young.']
['It is our choices, Harry, that show what we truly are, far more than our abilities.', 'Soon we must all face the choice between what is right and what is easy.']


In [ ]:
dataloader

Next step is to convert these sentences into tensors. Let's see how to do this.

In [13]:
sentences = [
    "If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals.",
    "Fame's a fickle friend, Harry.",
    "It is our choices, Harry, that show what we truly are, far more than our abilities.",
    "Soon we must all face the choice between what is right and what is easy.",
    "Youth can not know how age thinks and feels. But old men are guilty if they forget what it was to be young.",
    "You are awesome!"
]

# Define a custom data set
class CustomDataset(Dataset):
    def __init__(self, sentences, tokenizer, vocab):
        self.sentences = sentences
        self.tokenizer = tokenizer
        self.vocab = vocab

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        tokens = self.tokenizer(self.sentences[idx])
        # Convert tokens to tensor indices using vocab
        tensor_indices = [self.vocab[token] for token in tokens]
        return torch.tensor(tensor_indices)

# Tokenizer
tokenizer = get_tokenizer("basic_english")

# Build vocabulary
vocab = build_vocab_from_iterator(map(tokenizer, sentences))

# Create an instance of your custom data set
custom_dataset = CustomDataset(sentences, tokenizer, vocab)

print("Custom Dataset Length:", len(custom_dataset))
print("Sample Items:")
for i in range(6):
    sample_item = custom_dataset[i]
    print(f"Item {i + 1}: {sample_item}")

Custom Dataset Length: 6
Sample Items:
Item 1: tensor([11, 19, 63, 17, 13,  2,  3, 47,  6, 16, 45,  0, 55,  3, 41, 46, 24, 10,
        43, 61,  9, 44,  0, 14,  9, 33,  1])
Item 2: tensor([35,  6, 16,  3, 38, 40,  0,  8,  1])
Item 3: tensor([12,  5, 15, 31,  0,  8,  0, 57, 53,  2, 18, 62,  4,  0, 36, 49, 56, 15,
        21,  1])
Item 4: tensor([54, 18, 50, 23, 34, 58, 30, 27,  2,  5, 52,  7,  2,  5, 32,  1])
Item 5: tensor([66, 29, 14, 13, 10, 22, 60,  7, 37,  1, 28, 51, 48,  4, 42, 11, 59, 39,
         2, 12, 64, 17, 26, 65,  1])
Item 6: tensor([19,  4, 25, 20])


In [14]:
# Define batch size
batch_size = 2

# Create a data loader
dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Iterate through the data loader
for batch in dataloader:
    print(batch)

RuntimeError: stack expects each tensor to be equal size, but got [20] at entry 0 and [4] at entry 1

This error arises because the tensor batches have unequal lengths. The data loader is using the default collate_function, which requires tensors to have equal lengths. We can define our own collate_function and pass the data into it to establish your own rules. Typically, to address the issue of unequal tensor lengths,  employ data padding.

In [15]:
# Create a custom collate function
def collate_fn(batch):
    # Pad sequences within the batch to have equal lengths
    padded_batch = pad_sequence(batch, batch_first=True, padding_value=0)
    return padded_batch

In [16]:
# Create a data loader with the custom collate function with batch_first=True,
dataloader = DataLoader(custom_dataset, batch_size=batch_size, collate_fn=collate_fn)

# Iterate through the data loader
for batch in dataloader:
    for row in batch:
        for idx in row:
            words = [vocab.get_itos()[idx] for idx in row]
        print(words)


['if', 'you', 'want', 'to', 'know', 'what', 'a', 'man', "'", 's', 'like', ',', 'take', 'a', 'good', 'look', 'at', 'how', 'he', 'treats', 'his', 'inferiors', ',', 'not', 'his', 'equals', '.']
['fame', "'", 's', 'a', 'fickle', 'friend', ',', 'harry', '.', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',']
['it', 'is', 'our', 'choices', ',', 'harry', ',', 'that', 'show', 'what', 'we', 'truly', 'are', ',', 'far', 'more', 'than', 'our', 'abilities', '.']
['soon', 'we', 'must', 'all', 'face', 'the', 'choice', 'between', 'what', 'is', 'right', 'and', 'what', 'is', 'easy', '.', ',', ',', ',', ',']
['youth', 'can', 'not', 'know', 'how', 'age', 'thinks', 'and', 'feels', '.', 'but', 'old', 'men', 'are', 'guilty', 'if', 'they', 'forget', 'what', 'it', 'was', 'to', 'be', 'young', '.']
['you', 'are', 'awesome', '!', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',']


In [17]:
# Iterate through the data loader with batch_first = TRUE
for batch in dataloader:
    print(batch)
    print("Length of sequences in the batch:",batch.shape[1])

tensor([[11, 19, 63, 17, 13,  2,  3, 47,  6, 16, 45,  0, 55,  3, 41, 46, 24, 10,
         43, 61,  9, 44,  0, 14,  9, 33,  1],
        [35,  6, 16,  3, 38, 40,  0,  8,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0]])
Length of sequences in the batch: 27
tensor([[12,  5, 15, 31,  0,  8,  0, 57, 53,  2, 18, 62,  4,  0, 36, 49, 56, 15,
         21,  1],
        [54, 18, 50, 23, 34, 58, 30, 27,  2,  5, 52,  7,  2,  5, 32,  1,  0,  0,
          0,  0]])
Length of sequences in the batch: 20
tensor([[66, 29, 14, 13, 10, 22, 60,  7, 37,  1, 28, 51, 48,  4, 42, 11, 59, 39,
          2, 12, 64, 17, 26, 65,  1],
        [19,  4, 25, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0]])
Length of sequences in the batch: 25


each batch has a fixed size for all the sequences within the batch.



We also have the option to utilize the collate function for tasks such as tokenization, converting tokenized indices, and transforming the result into a tensor. It's important to note that the original data set remains untouched by these transformations.

In [18]:
# Define a custom data set
class CustomDataset(Dataset):
    def __init__(self, sentences):
        self.sentences = sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx]

In [19]:
custom_dataset=CustomDataset(sentences)

In [20]:
custom_dataset[0]

"If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals."

In [21]:
def collate_fn(batch):
    # Tokenize each sample in the batch using the specified tokenizer
    tensor_batch = []
    for sample in batch:
        tokens = tokenizer(sample)
        # Convert tokens to vocabulary indices and create a tensor for each sample
        tensor_batch.append(torch.tensor([vocab[token] for token in tokens]))

    # Pad sequences within the batch to have equal lengths using pad_sequence
    # batch_first=True ensures that the tensors have shape (batch_size, max_sequence_length)
    padded_batch = pad_sequence(tensor_batch, batch_first=True)

    # Return the padded batch
    return padded_batch

In [22]:
# Create a data loader for the custom dataset
dataloader = DataLoader(
    dataset=custom_dataset,   # Custom PyTorch Dataset containing your data
    batch_size=batch_size,     # Number of samples in each mini-batch
    shuffle=True,              # Shuffle the data at the beginning of each epoch
    collate_fn=collate_fn      # Custom collate function for processing batches
)

In [23]:
for batch in dataloader:
    print(batch)
    print("shape of sample",len(batch))

tensor([[11, 19, 63, 17, 13,  2,  3, 47,  6, 16, 45,  0, 55,  3, 41, 46, 24, 10,
         43, 61,  9, 44,  0, 14,  9, 33,  1],
        [54, 18, 50, 23, 34, 58, 30, 27,  2,  5, 52,  7,  2,  5, 32,  1,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0]])
shape of sample 2
tensor([[35,  6, 16,  3, 38, 40,  0,  8,  1],
        [19,  4, 25, 20,  0,  0,  0,  0,  0]])
shape of sample 2
tensor([[66, 29, 14, 13, 10, 22, 60,  7, 37,  1, 28, 51, 48,  4, 42, 11, 59, 39,
          2, 12, 64, 17, 26, 65,  1],
        [12,  5, 15, 31,  0,  8,  0, 57, 53,  2, 18, 62,  4,  0, 36, 49, 56, 15,
         21,  1,  0,  0,  0,  0,  0]])
shape of sample 2


a data loader with a collate function that processes batches of French text (provided below). Sort the data set on sequences length. Then tokenize, numericalize and pad the sequences. Sorting the sequences will minimize the number of <PAD>tokens added to the sequences, which enhances the model's performance. Prepare the data in batches of size 4 and print them.

In [39]:

# Example French corpus
corpus = [
    "Ceci est une phrase.",
    "C'est un autre exemple de phrase.",
    "Voici une troisième phrase.",
    "Il fait beau aujourd'hui.",
    "J'aime beaucoup la cuisine française.",
    "Quel est ton plat préféré ?",
    "Je t'adore.",
    "Bon appétit !",
    "Je suis en train d'apprendre le français.",
    "Nous devons partir tôt demain matin.",
    "Je suis heureux.",
    "Le film était vraiment captivant !",
    "Je suis là.",
    "Je ne sais pas.",
    "Je suis fatigué après une longue journée de travail.",
    "Est-ce que tu as des projets pour le week-end ?",
    "Je vais chez le médecin cet après-midi.",
    "La musique adoucit les mœurs.",
    "Je dois acheter du pain et du lait.",
    "Il y a beaucoup de monde dans cette ville.",
    "Merci beaucoup !",
    "Au revoir !",
    "Je suis ravi de vous rencontrer enfin !",
    "Les vacances sont toujours trop courtes.",
    "Je suis en retard.",
    "Félicitations pour ton nouveau travail !",
    "Je suis désolé, je ne peux pas venir à la réunion.",
    "À quelle heure est le prochain train ?",
    "Bonjour !",
    "C'est génial !"
]

In [41]:
from transformers import CamembertTokenizer
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

# Load the Camembert tokenizer
camembert_tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# Define a function to tokenize and convert tokens to IDs
def tokenize(sentence):
    return camembert_tokenizer.encode(sentence, add_special_tokens=False)


# Define a collate function for padding
def collate_fn_fr(batch):
    tensor_batch = []
    for sample in batch:
        tokens = tokenize(sample)
        tensor_batch.append(torch.tensor(tokens))
    padded_batch = pad_sequence(tensor_batch, batch_first=True)
    return padded_batch

# Sort sentences based on their length
sorted_data = sorted(corpus, key=lambda x: len(tokenize(x)))


# Create a DataLoader
dataloader = DataLoader(sorted_data, batch_size=2, shuffle=False, collate_fn=collate_fn_fr)

# Iterate through the DataLoader
for batch in dataloader:
    print(batch)


tensor([[ 1285,    83,     0],
        [  965, 18956,    83]])
tensor([[ 757,  217,   83],
        [ 277, 3519,   83]])
tensor([[ 100,  146, 1941,    9],
        [ 100,  146,  241,    9]])
tensor([[2978,   30,   28, 3572,    9],
        [1119,   28, 1390, 3572,    9]])
tensor([[ 100,  271,   11, 1990,    9],
        [ 100,   45,  555,   34,    9]])
tensor([[ 100,  146,   22, 2599,    9],
        [  84,   11,   41, 6023,   83]])
tensor([[ 2302,    30,   415,  1900,  3503,   106],
        [   54,   492,   149,   302, 17428,    83]])
tensor([[22734,    24,   415,   281,   225,    83,     0],
        [   69,    82,   670,   405,    11,   265,     9]])
tensor([[  170,  4092,   350,  2086,  2385,   823,     9],
        [   61,   680, 20328,   110,    19, 20722,     9]])
tensor([[  74,  866,   56,  179,  237, 8072,    9,    0],
        [ 121,   11,  660,  217,   13,  516,  781,    9]])
tensor([[ 100,  146, 4442,    8,   39, 1162,  743,   83],
        [ 468, 1262, 1507,   30,   16, 1319, 1062,

In [40]:
corpus

['Ceci est une phrase.',
 "C'est un autre exemple de phrase.",
 'Voici une troisième phrase.',
 "Il fait beau aujourd'hui.",
 "J'aime beaucoup la cuisine française.",
 'Quel est ton plat préféré ?',
 "Je t'adore.",
 'Bon appétit !',
 "Je suis en train d'apprendre le français.",
 'Nous devons partir tôt demain matin.',
 'Je suis heureux.',
 'Le film était vraiment captivant !',
 'Je suis là.',
 'Je ne sais pas.',
 'Je suis fatigué après une longue journée de travail.',
 'Est-ce que tu as des projets pour le week-end ?',
 'Je vais chez le médecin cet après-midi.',
 'La musique adoucit les mœurs.',
 'Je dois acheter du pain et du lait.',
 'Il y a beaucoup de monde dans cette ville.',
 'Merci beaucoup !',
 'Au revoir !',
 'Je suis ravi de vous rencontrer enfin !',
 'Les vacances sont toujours trop courtes.',
 'Je suis en retard.',
 'Félicitations pour ton nouveau travail !',
 'Je suis désolé, je ne peux pas venir à la réunion.',
 'À quelle heure est le prochain train ?',
 'Bonjour !',
 "C'

### Special symbols
In a typical NLP  context, the tokens `['<unk>', '<pad>', '<bos>', '<eos>']` have specific meanings:

1. `<unk>`: This token represents "unknown" or "out-of-vocabulary" words. It is used when a word in the input text is not found in the vocabulary or when dealing with words that are rare or unseen during training. When the model encounters an unknown word, it replaces it with the `<unk>` token.

2. `<pad>`: This token represents padding. In sequences of text data, such as sentences or documents, sequences may have different lengths. To create batches of data with uniform dimensions, shorter sequences are often padded with this `<pad>` token to match the length of the longest sequence in the batch.

3. `<bos>`: This token represents the "beginning of sequence." It is used to indicate the start of a sentence or sequence of tokens. It helps the model understand the beginning of a text sequence.

4. `<eos>`: This token represents the "end of sequence." It is used to indicate the end of a sentence or sequence of tokens. It helps the model recognize the end of a text sequence.

 Define special symbols and indices
